In [ ]:
!pip install gensim
!pip install transformers
!pip install tensorflow-text==2.9.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 59.4 MB/s eta 0:00:00


In [ ]:
import gensim
import numpy
from gensim import corpora, models, similarities
import jieba
import pandas as pd                                                        #data import
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer                  #character tokenization
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences          #padding
from tqdm import tqdm                                                      #track progress
import numpy as np                                                         #for numpy operations
from sklearn.model_selection import train_test_split                       #to split the data into train and test sets
from copy import deepcopy
from numpy import save, load
from transformers import BertTokenizer
import tensorflow_hub as hub
import tensorflow_text as text
import pickle
import csv
nltk.download('punkt')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

MessageError: ignored

In [ ]:
d1 = pd.read_csv("/content/question_answers.csv")
d1.head()

FileNotFoundError: ignored

In [ ]:
d2 = pd.read_csv("/content/paragraphs.csv")
d2.head()

In [ ]:
#data = pd.read_csv("question_answers.csv")
#data.head()

In [ ]:
#data2 = data.drop(['paragraph_id','answer'],axis=1)
#data2.to_csv('question_data.csv', index=False)

In [ ]:
lst = list(csv.DictReader(open('paragraphs.csv')))
ques = list(csv.DictReader(open('question_answers.csv')))

In [ ]:
lst[25]

In [ ]:
ques[943]

In [ ]:
para_id = []
Paragraph=[]
for i in range(len(ques)):
  texts = []
  for j in range(len(lst)):
    if(lst[j]['theme'] == ques[i]['theme']):
      texts.append(lst[j]['paragraph'])
  keyword = ques[i]['question']
  texts1 = [jieba.lcut(text) for text in texts]
  dictionary = corpora.Dictionary(texts1)
  feature_cnt = len(dictionary.token2id)
  corpus = [dictionary.doc2bow(text) for text in texts1]
  tfidf = models.TfidfModel(corpus) 
  kw_vector = dictionary.doc2bow(jieba.lcut(keyword))
  index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = feature_cnt)
  sim = index[tfidf[kw_vector]]
  ind = sim.argmax()
  Paragraph.append(texts[ind])
  para_id.append(ind+1)

In [ ]:
d1 = d1.assign(Paragraph=Paragraph)
d1 = d1.assign(ID=para_id)

In [ ]:
def tokenize_entries(row):
    """Function that tokenizes(both word and character level) the question and context of all records"""
    # row['word_tokens'] = tokenizer(row['Paragraph'],row['Question'],max_length=512,truncation='longest_first')
    ctok = tokenizer(row['Paragraph'])
    qtok = tokenizer(row['Question'])

    ctok['input_ids'] = ctok['input_ids'] + [0]*461
    ctok['input_ids'] = ctok['input_ids'][0:460]+[120]
    ctok['attention_mask'] = ctok['attention_mask'] + [0]*461
    ctok['attention_mask'] = ctok['attention_mask'][0:460]+[1]
    ctok['token_type_ids'] = ctok['token_type_ids'] + [0]*461
    ctok['token_type_ids'] = ctok['token_type_ids'][0:461]

    # print(len(ctok['token_type_ids']))
    # print(len(ctok['input_ids']))
    # print(len(ctok['attention_mask']))

    # print(qtok['input_ids'][0])
    qtok['input_ids'] = qtok['input_ids'][1:] + [0]*51
    qtok['input_ids'] = qtok['input_ids'][0:51]
    # print(qtok['input_ids'][0])
    qtok['attention_mask'] = qtok['attention_mask'][1:] + [0]*51
    qtok['attention_mask'] = qtok['attention_mask'][0:51]
    qtok['token_type_ids'] = [1]*51

    # print(len(qtok['token_type_ids']))
    # print(len(qtok['input_ids']))
    # print(len(qtok['attention_mask']))

    ctok['input_ids'] = ctok['input_ids'] +qtok['input_ids']
    ctok['attention_mask'] = ctok['attention_mask'] + qtok['attention_mask'] 
    ctok['token_type_ids'] = ctok['token_type_ids'] + qtok['token_type_ids'] 

    # print(len(ctok['token_type_ids']))
    # print(len(ctok['input_ids']))
    # print(len(ctok['attention_mask']))
    row['word_tokens'] = ctok
    
    return row

In [ ]:
d1

,question,theme,paragraph_id,answer,Paragraph,ID
0,Which company produces the iPod?,IPod,1,Apple,"Though the iPod was released in 2001, its pric...",6
1,When was the original iPod released?,IPod,1,"October 23, 2001","The games are in the form of .ipg files, which...",34
2,How many different types of iPod are currently...,IPod,1,three,"Unlike many other MP3 players, simply copying ...",35
3,What kind of device is the iPod?,IPod,1,portable media players,Besides earning a reputation as a respected en...,47
4,The iPod Touch uses what kind of interface?,IPod,1,touchscreen,The dock connector also allowed the iPod to co...,17
...,...,...,...,...,...,...
939,What type of songs was Leona Lewis known for?,Adult_contemporary_music,640,ballads,"Over the years, AC has spawned subgenres inclu...",5
940,"Along with mainstream AC, what format has also...",Adult_contemporary_music,641,hot AC,Contemporary Christian music (CCM) has several...,50
941,What AC format is especially known for playing...,Adult_contemporary_music,642,soft AC,In recent years it has become common for many ...,51
942,When is the later of the two times that Christ...,Adult_contemporary_music,643,New Year's Day,The amount of crossover between the AC chart a...,19


In [ ]:
d1.rename(columns = {'question':'Question'}, inplace = True)

In [ ]:
d1.iloc[943]['Paragraph']

'In recent years it has become common for many AC stations, particularly soft AC stations, to play primarily or exclusively Christmas music during the Christmas season in November and December. While these tend mostly to be contemporary seasonal recordings by the same few artists featured under the normal format, most stations will also air some vintage holiday tunes from older pop, MOR, and adult standards artists – such as Nat King Cole, Bing Crosby, Dean Martin, The Carpenters, Percy Faith, Johnny Mathis and Andy Williams – many of whom would never be played on these stations during the rest of the year.'

In [ ]:
tqdm.pandas()

In [ ]:
df_final = d1.progress_apply(tokenize_entries,axis=1)

100%|██████████| 944/944 [00:03<00:00, 251.07it/s]


In [ ]:
df_final

,Question,theme,paragraph_id,answer,Paragraph,ID,word_tokens
0,Which company produces the iPod?,IPod,1,Apple,"Though the iPod was released in 2001, its pric...",6,"[input_ids, token_type_ids, attention_mask]"
1,When was the original iPod released?,IPod,1,"October 23, 2001","The games are in the form of .ipg files, which...",34,"[input_ids, token_type_ids, attention_mask]"
2,How many different types of iPod are currently...,IPod,1,three,"Unlike many other MP3 players, simply copying ...",35,"[input_ids, token_type_ids, attention_mask]"
3,What kind of device is the iPod?,IPod,1,portable media players,Besides earning a reputation as a respected en...,47,"[input_ids, token_type_ids, attention_mask]"
4,The iPod Touch uses what kind of interface?,IPod,1,touchscreen,The dock connector also allowed the iPod to co...,17,"[input_ids, token_type_ids, attention_mask]"
...,...,...,...,...,...,...,...
939,What type of songs was Leona Lewis known for?,Adult_contemporary_music,640,ballads,"Over the years, AC has spawned subgenres inclu...",5,"[input_ids, token_type_ids, attention_mask]"
940,"Along with mainstream AC, what format has also...",Adult_contemporary_music,641,hot AC,Contemporary Christian music (CCM) has several...,50,"[input_ids, token_type_ids, attention_mask]"
941,What AC format is especially known for playing...,Adult_contemporary_music,642,soft AC,In recent years it has become common for many ...,51,"[input_ids, token_type_ids, attention_mask]"
942,When is the later of the two times that Christ...,Adult_contemporary_music,643,New Year's Day,The amount of crossover between the AC chart a...,19,"[input_ids, token_type_ids, attention_mask]"


In [ ]:
len(df_final.iloc[0]['word_tokens']['attention_mask'])

512

In [ ]:
train_input_mask,train_input_ids,train_input_type_ids = [],[],[]
for row in df_final['word_tokens'].values:
  train_input_mask.append(row['attention_mask'])
  train_input_ids.append(row['input_ids'])
  train_input_type_ids.append(row['token_type_ids'])

In [ ]:
train_input_mask = np.array(train_input_mask,dtype = np.int32)
train_input_ids = np.array(train_input_ids,dtype = np.int32)
train_input_type_ids = np.array(train_input_type_ids,dtype = np.int32)

In [ ]:
save("/content/mask_test.npy", train_input_mask)
save("/content/ids_test.npy", train_input_ids)
save("/content/type_test.npy", train_input_type_ids)

In [ ]:
train_input_ids.shape

(944, 512)